In [ ]:
# check pytorch installation: 
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

In [3]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [4]:
# diffusiondet imports
import sys
sys.path.append('DiffusionDet')

from diffusiondet.util.model_ema import add_model_ema_configs, may_build_model_ema, may_get_ema_checkpointer, EMAHook, \
    apply_model_ema_and_restore, EMADetectionCheckpointer
from detectron2.checkpoint import DetectionCheckpointer

from train_net import Trainer
from diffusiondet import add_diffusiondet_config
from diffusiondet.util.model_ema import add_model_ema_configs

## Data preparation

In [8]:
# register datasets
from detectron2.data.datasets import register_coco_instances
register_coco_instances("iSAID_train", {}, 
                        "/share/sda/aleksandrmatsun/iSAID/iSAID_patches/train/instancesonly_filtered_train.json",
                        "/share/sda/aleksandrmatsun/iSAID/iSAID_patches/train/images/")
register_coco_instances("iSAID_val", {}, 
                        "/share/sda/aleksandrmatsun/iSAID/iSAID_patches/val/instancesonly_filtered_val.json",
                        "/share/sda/aleksandrmatsun/iSAID/iSAID_patches/val/images/")

In [ ]:
import cv2
import numpy as np
import random
from detectron2.data.transforms import RandomFlip as D2RandomFlip
from detectron2.data import transforms as T
from detectron2.data import DatasetMapper

# creating custom augmentation classes compatible with Detectron2 framework

class MyCustomResize(T.Augmentation):
    def get_transform(self, image):
        old_h, old_w = image.shape[:2]
        new_h, new_w = int(old_h * np.random.rand()), int(old_w * 1.5)
        return T.ResizeTransform(old_h, old_w, new_h, new_w)

class MyCustomRandomFlip(T.Augmentation):
    def __init__(self, horizontal=True, vertical=False):
        super().__init__()
        self.horizontal = horizontal
        self.vertical = vertical

    def get_transform(self, image):
        flip_transform = D2RandomFlip(horizontal=self.horizontal, vertical=self.vertical)
        return flip_transform.get_transform(image)

class AlbumentationsTransform(T.Transform):
    def __init__(self, albumentations_transform, p=1.0):
        super().__init__()
        self.albumentations_transform = albumentations_transform
        self.p = p

    def apply_image(self, image):
        if self.albumentations_transform and random.random() < self.p:
            transformed = self.albumentations_transform(image=image)
            if transformed is not None:
                return transformed['image']
        return image

class MyAlbumentations(T.Augmentation):
    def __init__(self, p=1.0):
        super().__init__()
        self.p = p
        self.transform = None
        try:
            import albumentations as A

            T = [
                A.Blur(p=0.01),
                A.MedianBlur(p=0.01),
                A.ToGray(p=0.01),
                A.CLAHE(p=0.01),
                A.RandomBrightnessContrast(p=0.0),
                A.RandomGamma(p=0.0),
                A.ImageCompression(quality_lower=75, p=0.0)]  # transforms
            self.transform = A.Compose(T, bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

        except ImportError:  # package not installed, skip
            pass

    def get_transform(self, image):
        return AlbumentationsTransform(self.transform, self.p)

def register_new_dataset():

    dataset_mapper = DatasetMapper(cfg, is_train=True, augmentations=[
        MyCustomResize(),
        MyCustomRandomFlip(horizontal=True, vertical=False),
        MyAlbumentations(p=1.0)  # Add custom Albumentations augmentation
    ])
    iSAID_train_metadata = MetadataCatalog.get("iSAID_train")
    iSAID_train_metadata.dataset_mapper = dataset_mapper

    return iSAID_train_metadata, MetadataCatalog.get("iSAID_val")

## Training preparation

In [ ]:
# backbone preparing
from detectron2.modeling import BACKBONE_REGISTRY, Backbone, ShapeSpec
from mmrotate.models.builder import ROTATED_BACKBONES, ROTATED_NECKS

In [10]:
# backbone class
@BACKBONE_REGISTRY.register()
class LSKBackbone(Backbone):
  def __init__(self, cfg, input_shape):
    super().__init__()
    # backbone definition
    self.bb = ROTATED_BACKBONES.get('LSKNet')(embed_dims=[64, 128, 320, 512], drop_rate=0.1, drop_path_rate=0.1, depths=[2,2,4,2])
    self.neck = ROTATED_NECKS.get('FPN')(in_channels=[64, 128, 320, 512],
        out_channels=256,
        num_outs=5)

  def forward(self, image):
    out = self.bb(image)
    out = self.neck(out)
    return {
      'p2' : out[0],
      'p3' : out[1],
      'p4' : out[2],
      'p5' : out[3],
      'p6' : out[4],
    }

  def output_shape(self):
    return {
      'p2': ShapeSpec(channels=256, stride=4),
      'p3': ShapeSpec(channels=256, stride=8),
      'p4': ShapeSpec(channels=256, stride=16),
      'p5': ShapeSpec(channels=256, stride=32),
      'p6': ShapeSpec(channels=256, stride=64),
    }

In [11]:
from detectron2.engine import DefaultTrainer
# config preparation
cfg = get_cfg()
cfg.OUTPUT_DIR = '/home/aleksandrmatsun/projectron/output_best' # the output directory should be changed accordingly
add_diffusiondet_config(cfg)
add_model_ema_configs(cfg)
cfg.merge_from_file("/home/aleksandrmatsun/projectron/DiffusionDet/configs/diffdet.coco.res50.yaml") # the config from DiffusionDet to be merged with
cfg.DATASETS.TRAIN = ("iSAID_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.DEVICE = "cuda:0"
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.00005  # small learning rate due to DiffusionDet properties
cfg.SOLVER.MAX_ITER = 100000
cfg.SOLVER.CHECKPOINT_PERIOD = 10000
cfg.MODEL.DiffusionDet.NUM_CLASSES = 15
cfg.MODEL.DiffusionDet.NUM_PROPOSALS = 700
cfg.MODEL.ANCHOR_GENERATOR.ASPECT_RATIOS = [[0.25, 0.75, 2.0, 4.0]] # setting custom aspect ratios
cfg.MODEL.BACKBONE.NAME = "LSKBackbone" # setting the LSKNet as backbone

cfg.MODEL.BACKBONE.FREEZE_AT = 0 # this line is redundant, but kept for consistency


In [ ]:
# register_new_dataset() # applying custom augmentations

In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = Trainer(cfg) # creating the trainer

In [15]:
w = torch.load('/home/aleksandrmatsun/projectron/weights/lsk_s_backbone-e9d2e551.pth') # weights for backbone (pretrained on imagenet)
w2 = torch.load('/home/aleksandrmatsun/projectron/weights/diffdet_coco_res101.pth') # weights for heads (pretrained on COCO)

In [16]:
w_heads = {k:w2['model'][k] for k in w2['model'] if (not k.startswith('backbone') and k.find('class_logits')==-1)}

In [17]:
# the neck of the model is initialized from scratch, due to unavailability of any fitting weights
trainer.model.backbone.bb.load_state_dict(w['state_dict'], strict=False) 
trainer.model.load_state_dict(w_heads, strict=False)
0

0

In [ ]:
# training function
trainer.train()

## Evaluation

In [ ]:
register_coco_instances("iSAID_test", {}, 
                        "/share/sda/aleksandrmatsun/iSAID/iSAID_patches/test/test_info.json",
                        "/share/sda/aleksandrmatsun/iSAID/iSAID_patches/test/images/")

In [18]:
cfg.MODEL.WEIGHTS = os.path.join('/home/aleksandrmatsun/projectron/output_best/model_0099999.pth') # weights to be evaluated

In [19]:
cfg.DATASETS.TEST = ("iSAID_val",)  # for evaluation on validation subset
# cfg.DATASETS.TEST = ("iSAID_test",) # for evaluation on test subset

In [20]:
m = Trainer.build_model(cfg)
kwargs = may_get_ema_checkpointer(cfg, m)
cfg.MODEL_EMA

In [ ]:
DetectionCheckpointer(m, save_dir=cfg.OUTPUT_DIR, **kwargs).resume_or_load(cfg.MODEL.WEIGHTS,
                                                                                           resume=True) # checkpointer creation

In [ ]:
res = Trainer.ema_test(cfg, m) # no EMA is actually used, this function runs regular evaluation

In [ ]:
res